In [7]:
import tensorflow as tf
import numpy as np
import math
import glob
import matplotlib

import matplotlib.pyplot as plt
from matplotlib.pyplot import plot,savefig
from dataset import *

from sklearn.datasets import load_iris
import xgboost as xgb
import sklearn as sk
from xgboost import plot_importance
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import mylib as ml2
from mylib import LiftNet, create_LiftNet, create_Standard_LiftNet, Standard_LiftNet, create_Standard_LiftNet_CWRU

from plotly.graph_objs import Scatter,Layout
import plotly
import plotly.offline as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.subplots import make_subplots
import pandas as pd
import plotly.figure_factory as ff

from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_validate
import pandas as pd

/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [26]:
test_rate = 0.1
lr=0.015
momentum=0.8
decay=0.01
validation_split=0.2
validation_steps=1
epochs = 10000

LiftingNet_noise_scale = 0.2
train_steps = 800
steps_per_epoch=1
cutsize = 1024

class_num = 6
bunch_steps = 100
snapshot = 500


circle_num =1


whether_expansion_data = 1
expansion_data_number = 500
noise_scales = 0.01


model_head_name = '/home/silver-bullet/newpaper/code1216/snapshot/snapshot_CWRU_LiftingNet/Standard_expansion_CWRU_data_LiftingNet_'
model_name = model_head_name + str(LiftingNet_noise_scale) + '_data_the_' + str(train_steps) + 'th_snapshot_with_' + str(steps_per_epoch) + '_steps_per_epoch.h5'

data_path = '/home/silver-bullet/newpaper/data/CWRUdataset'
dataset, label = load_CWRU_data(data_path)

if whether_expansion_data == 1:
    dataset, label = expansion_and_add_noise(dataset,label,exnumber=expansion_data_number, noise_scales=noise_scales)


dataset shape :  (832, 1024, 2)
label shape :  (832, 1)
(237, 1024, 2)
(118, 1024, 2)
(117, 1024, 2)
(118, 1024, 2)
(118, 1024, 2)
(118, 1024, 2)
(500, 1024, 2)
(500, 1)
(500, 1024, 2)
(500, 1)
(500, 1024, 2)
(500, 1)
(500, 1024, 2)
(500, 1)
(500, 1024, 2)
(500, 1)
(500, 1024, 2)
(500, 1)
(3000, 1024, 2)
(3000, 1)
data_max_abs_x.shape:  (3000, 1, 2)
noise_data.shape:  (3000, 1024, 2)
after expansion: 
dataset shape :  (3832, 1024, 2)
label shape :  (3832, 1)


In [27]:
x_number = dataset.shape[0]
print(x_number)

dataset = dataset.astype(np.float32)
print(dataset.dtype)
print('dataset shape : ',dataset.shape)
print('label shape : ', label.shape)

input_shape = (dataset.shape[1],dataset.shape[2])
channel = dataset.shape[2]

select_feature_numbers = 30

artificial_feature_data = feature_extractor2(dataset)
#artificial_feature_data = artificial_feature_data.reshape(x_number,-1)[:,:select_feature_numbers]
artificial_feature_data = artificial_feature_data.reshape(x_number,-1)
print(artificial_feature_data.shape)

3832
float32
dataset shape :  (3832, 1024, 2)
label shape :  (3832, 1)
p1.shape:  (3832, 2)
p2.shape:  (3832, 2)
p3.shape:  (3832, 2)
p4.shape:  (3832, 2)
p5.shape:  (3832, 2)
p6.shape:  (3832, 2)
p7.shape:  (3832, 2)
p8.shape:  (3832, 2)
p9.shape:  (3832, 2)
(3832, 18)


In [28]:
liftnet = create_Standard_LiftNet_CWRU(class_num = class_num, channel = channel, cut_size = cutsize, input_shape=input_shape,lr=lr, momentum=momentum, decay=decay)
liftnet.load_weights(model_name)
print('load model')

feature_data = liftnet.feature_extractor(dataset)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    feature_data = feature_data.eval()
#feature_data = tf.Session().run(feature_data)
#sess = tf.InteractiveSession()
#feature_data = feature_data.eval()
#print(feature_data)
print('feature_data.shape: ', feature_data.shape)
feature_data = np.reshape(feature_data,(feature_data.shape[0],feature_data.shape[1]))
#feature_data = np.concatenate(dataset,axis=2)
print(feature_data.shape)
print('finished feature extract')

pca_feature_extractor = PCA(n_components=18)
pca_feature_extractor.fit(feature_data)
pca_feature = pca_feature_extractor.transform(feature_data)
print(pca_feature_extractor.n_components_)
print(pca_feature_extractor.explained_variance_ratio_)

print(pca_feature.shape)

x = pca_feature.reshape(x_number,-1)


Model: "LiftNet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 1024, 2)]         0         
_________________________________________________________________
convinputs (Conv1D)          multiple                  6         
_________________________________________________________________
predict1_1 (Conv1D)          multiple                  14        
_________________________________________________________________
predict1_2 (Conv1D)          multiple                  30        
_________________________________________________________________
update1_1 (Conv1D)           multiple                  62        
_________________________________________________________________
update1_2 (Conv1D)           multiple                  30        
_________________________________________________________________
predict2_1 (Conv1D)          multiple                  310 

In [29]:
#x = feature_data.reshape(x_number,-1)
x = np.concatenate((x,artificial_feature_data),axis=1)
print('x.shape: ',x.shape)


y = label.reshape(x_number,)
print('y.shape: ', y.shape)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=test_rate, random_state=99)

print('X_train.shape: ', X_train.shape)
print('X_test.shape: ',X_test.shape)
print('y_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)

x.shape:  (3832, 36)
y.shape:  (3832,)
X_train.shape:  (3448, 36)
X_test.shape:  (384, 36)
y_train.shape:  (3448,)
y_test.shape:  (384,)


In [30]:
model = XGBClassifier(learning_rate=0.1,
                        n_estimators=1000,         # 树的个数--1000棵树建立xgboost
                        max_depth=8,               # 树的深度
                        min_child_weight = 1,      # 叶子节点最小权重
                        gamma=0.1,                  # 惩罚项中叶子结点个数前的参数
                        subsample=0.8,             # 随机选择80%样本建立决策树
                        colsample_btree=0.8,       # 随机选择80%特征建立决策树
                        objective='multi:softmax', # 指定损失函数
                        scale_pos_weight=1,        # 解决样本个数不平衡的问题
                        )

model.fit(X_train,y_train)
# 对测试集进行预测
#dtest = xgb.DMatrix(X_test)
#ans = model.predict(dtest)
ans = model.predict(X_test)
print('ans.shape: ', ans.shape)


ml2.evaluate_model2(ans, y_test)

# 计算准确率
cnt1 = 0
cnt2 = 0

for i in range(len(y_test)):
    if ans[i] == y_test[i]:
        cnt1 += 1
    else:
        cnt2 += 1
print("Accuracy: %.2f %% " % (100 * cnt1 / (cnt1 + cnt2)))


ans.shape:  (384,)
normal result:
tp: 71  tn: 0  fp: 0  fn: 313
tp/(tp+tn):  1.0
tp/(tp+fp):  1.0
inner_ring result:
tp: 48  tn: 0  fp: 1  fn: 335
tp/(tp+tn):  1.0
tp/(tp+fp):  0.9795918367346939
outer_ring result:
tp: 67  tn: 0  fp: 0  fn: 317
tp/(tp+tn):  1.0
tp/(tp+fp):  1.0
roller result:
tp: 66  tn: 0  fp: 0  fn: 318
tp/(tp+tn):  1.0
tp/(tp+fp):  1.0
joint result:
tp: 50  tn: 1  fp: 0  fn: 333
tp/(tp+tn):  0.9803921568627451
tp/(tp+fp):  1.0
Accuracy: 99.48 % 


In [31]:
#---------------------------K Fold-------------------------------------------#
scoring = ['precision_macro', 'recall_macro']
scores = cross_validate(model, X_train, y_train, scoring=scoring,cv=10, return_train_score=True)
sorted(scores.keys())
print('test result:')
print(scores)
scores_df = pd.DataFrame(scores)

table_header = ['test_recall_macro', 'train_recall_macro','fit_time', 'train_precision_macro','test_precision_macro']

scores_df

test result:
{'train_precision_macro': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]), 'test_precision_macro': array([0.99707602, 1.        , 0.99404002, 1.        , 1.        ,
       1.        , 0.99404762, 0.99712644, 0.99712644, 0.99712644]), 'fit_time': array([13.38010383, 13.28495574, 12.99477887, 13.18203425, 13.32385278,
       13.51523328, 13.08314133, 13.23658061, 13.24838877, 13.800699  ]), 'test_recall_macro': array([0.99691358, 1.        , 0.99388328, 1.        , 1.        ,
       1.        , 0.99404572, 0.99691358, 0.99685535, 0.99685535]), 'score_time': array([0.05414057, 0.05735016, 0.04951906, 0.05421567, 0.0493238 ,
       0.04933429, 0.05040956, 0.05186629, 0.04876661, 0.04947686]), 'train_recall_macro': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])}


,fit_time,score_time,test_precision_macro,test_recall_macro,train_precision_macro,train_recall_macro
0,13.380104,0.054141,0.997076,0.996914,1.0,1.0
1,13.284956,0.057350,1.000000,1.000000,1.0,1.0
2,12.994779,0.049519,0.994040,0.993883,1.0,1.0
3,13.182034,0.054216,1.000000,1.000000,1.0,1.0
4,13.323853,0.049324,1.000000,1.000000,1.0,1.0
5,13.515233,0.049334,1.000000,1.000000,1.0,1.0
6,13.083141,0.050410,0.994048,0.994046,1.0,1.0
7,13.236581,0.051866,0.997126,0.996914,1.0,1.0
8,13.248389,0.048767,0.997126,0.996855,1.0,1.0
9,13.800699,0.049477,0.997126,0.996855,1.0,1.0
